In [8]:
import os
from pathlib import Path
import sys

sys.path.append("..")

import numpy as np
import torch

from models.experimental import attempt_load
from utils.datasets import LoadImages
from utils.general import check_img_size, non_max_suppression
from utils.torch_utils import select_device

In [12]:
data_dir = Path.cwd().parents[1] / "oidv6_data_converted_11classes"
count = 0
for path in (data_dir / "images" / "train").glob("*.jpg"):
    if (data_dir / "labels" / "train" / f"{path.stem}.txt").exists():
        with open(data_dir / "labels" / "train" / f"{path.stem}.txt", "r") as f:
            l = [x.split() for x in f.read().strip().splitlines()]
            l = np.array(l, dtype=np.float32)
        if not len(l):
            os.remove(path)
            os.remove(data_dir / "labels" / "train" / f"{path.stem}.txt")
            print(path)
        else:
            count += 1
print(count)

c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\cart_scraped_42.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\cart_scraped_45.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\gas_cylinder_scraped_102.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\gas_cylinder_scraped_47.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\gas_cylinder_scraped_91.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\table_scraped_224.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\table_scraped_251.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\table_scraped_82.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\tissue_scraped_130.jpg
c:\GitRepos\yolov5_workspace\oidv6_data_converted_11classes\images\train\tissue_scraped_131.jpg
43076


In [35]:
device = select_device("")
source = Path.cwd().parent / "data" / "images"
weights = Path.cwd().parent / "weights" / "yolov5l_freeze_11classes_scraped.pt"
imgsz = 640

conf_thres = 0.15
iou_thres = 0.45
classes = None
agnostic_nms = False

In [11]:
model = attempt_load(weights, map_location=device)  # load FP32 model


In [15]:
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size
names = model.module.names if hasattr(model, "module") else model.names  # get class names

In [23]:
dataset = LoadImages(source, img_size=imgsz, stride=stride)

In [39]:
for path, img, im0s, vid_cap in dataset:
    img = torch.from_numpy(img).to(device)
    img = img.float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    pred = model(img, augment=False)[0]
    pred_nms = non_max_suppression(pred, 0.001, 0.6, classes, agnostic_nms)
    break

image 1/24 c:\GitRepos\yolov5_workspace\yolov5\data\images\burner-cart-wok-1.jpg: tensor([4.39330e+00, 6.24763e+00, 1.14381e+01, 1.29285e+01, 7.94411e-06, 1.03331e-01, 3.34693e-02, 4.76894e-02, 4.16172e-02, 1.13475e-02, 1.18452e-02, 2.34487e-01, 1.95195e-02, 1.69694e-02, 5.56443e-02, 1.47641e-02])
[tensor([[4.05101e+02, 2.70091e+02, 5.60650e+02, 3.35862e+02, 7.25105e-01, 4.00000e+00],
        [1.11106e+02, 2.46666e+02, 4.99862e+02, 4.82934e+02, 6.19453e-01, 1.00000e+01],
        [4.38403e+02, 3.23211e+02, 5.30273e+02, 4.12509e+02, 5.38645e-01, 9.00000e+00],
        [1.23770e+02, 3.00477e+02, 1.54705e+02, 3.15651e+02, 3.57982e-01, 1.00000e+00]])]
